Homography estimation for OpenCV and Kornia DoG implementations (and their fixes)
Done on 4 datasets:

* boat and bark datasets from https://www.robots.ox.ac.uk/~vgg/data/affine/
* pure rotations of the first image from "bark" by multiples of 90 degrees
* pure scaling of the first image from "bark" by multiples of 0.1 adjusted so that the aspect ratio is preserved 

In [1]:
import cv2 as cv
from opencv_utils import get_tentatives, get_visible_part_mean_absolute_reprojection_error
from dataset_utils import Hs_imgs_for_boat, Hs_imgs_for_bark, Hs_imgs_for_rotation, Hs_imgs_for_scaling


def homography_estimation_experiment(detector, Hs_gt, imgs, e_name, instance_names):

    print(f"\nrunning experiment: {e_name}")
    print()
    print("\terror \t\t tentatives \t inliers")

    kpts_0, desc_0 = detector.detectAndCompute(imgs[0], mask=None)

    sum_reproj_err = 0.0
    sum_tent_count = 0
    sum_in_count = 0

    for other_i in range(1, len(imgs)):
        kpts_other, desc_other = detector.detectAndCompute(imgs[other_i], mask=None)

        src_pts, dst_pts = get_tentatives(kpts_0, desc_0, kpts_other, desc_other, ratio_threshold=0.8)
        if len(src_pts) < 4:
            print(f"WARNING: less than 4 tentatives: {len(src_pts)}")
            continue

        H_est, inlier_mask = cv.findHomography(src_pts, dst_pts,
                                               cv.RANSAC,
                                               maxIters=100000,
                                               ransacReprojThreshold=0.5,
                                               confidence=0.9999)
        H_gt = Hs_gt[other_i - 1]

        reproj_err = get_visible_part_mean_absolute_reprojection_error(imgs[0], imgs[other_i], H_gt, H_est)
        tent_count = len(src_pts)
        in_count = int(inlier_mask.sum())

        print(f"{instance_names[other_i - 1]}\t{reproj_err:.6f} \t {tent_count} \t\t {in_count}")
        sum_reproj_err += reproj_err
        sum_tent_count += tent_count
        sum_in_count += in_count

    print(f"Sum:\t{sum_reproj_err:.6f} \t {sum_tent_count} \t\t {sum_in_count}")


Hs_boat, imgs_boat = Hs_imgs_for_boat()
Hs_bark, imgs_bark = Hs_imgs_for_bark()
Hs_rot, imgs_rot = Hs_imgs_for_rotation()
print("Exact scale adjustments:")
Hs_scaling, imgs_scaling, scales = Hs_imgs_for_scaling()


Exact scale adjustments:
scale: 0.2 => 0.2
scale: 0.3 => 0.2980392156862745
scale: 0.4 => 0.4
scale: 0.5 => 0.5019607843137255
scale: 0.6 => 0.6
scale: 0.7 => 0.6980392156862745
scale: 0.8 => 0.8
scale: 0.9 => 0.9019607843137255


In [2]:
descriptor = cv.SIFT_create()

homography_estimation_experiment(descriptor, Hs_rot, imgs_rot, "synthetic pi rotation", ["90deg", "180deg", "270deg"])
homography_estimation_experiment(descriptor, Hs_bark, imgs_bark, "bark", [f"img{i}" for i in range(2, 7)])
homography_estimation_experiment(descriptor, Hs_boat, imgs_boat, "boat", [f"img{i}" for i in range(2, 7)])
homography_estimation_experiment(descriptor, Hs_scaling, imgs_scaling, "synthetic rescaling lanczos", [f"{s}" for s in scales])


running experiment: synthetic pi rotation

	error 		 tentatives 	 inliers
90deg	0.499982 	 3464 		 3452
180deg	0.704858 	 3369 		 3349
270deg	0.499913 	 3438 		 3430
Sum:	1.704754 	 10271 		 10231

running experiment: bark

	error 		 tentatives 	 inliers
img2	1.332286 	 654 		 241
img3	2.178040 	 565 		 298
img4	1.339833 	 699 		 620
img5	0.711655 	 490 		 433
img6	1.256951 	 305 		 261
Sum:	6.818765 	 2713 		 1853

running experiment: boat

	error 		 tentatives 	 inliers
img2	0.313058 	 2468 		 990
img3	0.285631 	 1838 		 926
img4	1.327328 	 757 		 241
img5	0.459227 	 550 		 140
img6	5.696269 	 267 		 63
Sum:	8.081513 	 5880 		 2360

running experiment: synthetic rescaling lanczos

	error 		 tentatives 	 inliers
0.2	0.283495 	 181 		 176
0.3	0.261653 	 391 		 374
0.4	0.212034 	 675 		 658
0.5	0.173158 	 1059 		 1042
0.6	0.142213 	 1402 		 1383
0.7	0.116124 	 1784 		 1621
0.8	0.069288 	 2315 		 2281
0.9	0.037124 	 2663 		 2625
Sum:	1.295089 	 10470 		 10160


# Comparison of OpenCV baseline and fixed version

* see https://github.com/opencv/opencv/pull/23124

```
Without fix:                                              With fix:


running experiment: synthetic pi rotation

              error    tentatives       inliers                          error    tentatives       inliers
 90deg     0.516270          6322          4857             90deg     0.000651          6656          6617
180deg     0.674599          5908          3802            180deg     0.001429          4436          4223
270deg     0.454714          6170          3872            270deg     0.001259          6114          5935
   Sum     1.645583         18400         12531               Sum     0.003339         17206         16775


running experiment: bark

              error    tentatives       inliers                          error    tentatives       inliers
img2       1.257790           267            49              img2     1.379990           331            91
img3       2.212071           121            28              img3     1.492706           125            48
img4       1.350093           362           193              img4     1.001385           335           252
img5       0.710791           210            90              img5     0.699119           233           171
img6       1.352123            89            43              img6     0.987482            77            62
 Sum       6.882868          1049           403               Sum     5.560682          1101           624


running experiment: boat

              error   tentatives        inliers                          error      tentatives     inliers
img2       0.625754         1640            425              img2     0.257128            1806         628
img3       0.313360         1155            345              img3     0.387596            1130         417
img4       1.635418          374             90              img4     0.834250             373         106
img5       0.868519          193             42              img5     1.171891             227          59
img6       6.004359           63             16              img6     4.849470              55          16
 Sum       9.447409         3425            918               Sum     7.500336            3591        1226


running experiment: synthetic rescaling lanczos

           error      tentatives        inliers                          error     tentatives      inliers
0.2     0.337641              46             28               0.2     0.063100             82           69
0.3     0.270277             219            132               0.3     0.105695            285          192
0.4     0.316275             597            316               0.4     0.050815            697          565
0.5     0.191226            1155            622               0.5     0.017677           1311         1160
0.6     0.162396            1782            985               0.6     0.047982           2065         1616
0.7     0.229305            2390           1260               0.7     0.050292           2832         2089
0.8     0.112914            3756           2134               0.8     0.015694           4140         3694
0.9     0.164707            4431           2340               0.9     0.051795           5233         3939
Sum     1.784741           14376           7817               Sum     0.403049          16645        13324
```

In [4]:
from kornia.feature.integrated import SIFTFeature
from kornia.utils import image_to_tensor
import torch


class NumpyKorniaSiftDescriptor:

    def __init__(self, device=torch.device("cpu")):
        self.device = device
        self.sf = SIFTFeature(device=device)

    @staticmethod
    def cv_kpt_from_laffs_responses(laffs, responses):
        kpts = []
        for i, response in enumerate(responses[0]):
            yx = laffs[0, i, :, 2]
            kp = cv.KeyPoint(yx[0].item(), yx[1].item(), response.item(), angle=0)
            kpts.append(kp)
        return kpts

    def detectAndCompute(self, img, mask):
        assert mask is None, "not implemented with non-trivial mask"
        if len(img.shape) == 2:
            img = img[:, :, None]
        else:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img_t = (image_to_tensor(img, False).float() / 255.).to(device=self.device)
        laffs, responses, descs = self.sf(img_t, mask=None)
        kpts = self.cv_kpt_from_laffs_responses(laffs, responses)
        descs = descs[0].cpu().detach().numpy()
        return kpts, descs


descriptor = NumpyKorniaSiftDescriptor()

homography_estimation_experiment(descriptor, Hs_rot, imgs_rot, "synthetic pi rotation", ["90deg", "180deg", "270deg"])
homography_estimation_experiment(descriptor, Hs_bark, imgs_bark, "bark", [f"img{i}" for i in range(2, 7)])
homography_estimation_experiment(descriptor, Hs_boat, imgs_boat, "boat", [f"img{i}" for i in range(2, 7)])
homography_estimation_experiment(descriptor, Hs_scaling, imgs_scaling, "synthetic rescaling lanczos", [f"{s}" for s in scales])



running experiment: synthetic pi rotation

	error 		 tentatives 	 inliers
90deg	0.516270 	 6322 		 4857
180deg	0.674599 	 5908 		 3802
270deg	0.454714 	 6170 		 3872
Sum:	1.645583 	 18400 		 12531

running experiment: bark

	error 		 tentatives 	 inliers
img2	1.257790 	 267 		 49
img3	2.212071 	 121 		 28
img4	1.350093 	 362 		 193
img5	0.710791 	 210 		 90
img6	1.352123 	 89 		 43
Sum:	6.882868 	 1049 		 403

running experiment: boat

	error 		 tentatives 	 inliers
img2	0.625754 	 1640 		 425
img3	0.313360 	 1155 		 345
img4	1.635418 	 374 		 90
img5	0.868519 	 193 		 42
img6	6.004359 	 63 		 16
Sum:	9.447409 	 3425 		 918

running experiment: synthetic rescaling lanczos

	error 		 tentatives 	 inliers
0.2	0.337641 	 46 		 28
0.3	0.270277 	 219 		 132
0.4	0.316275 	 597 		 316
0.5	0.191226 	 1155 		 622
0.6	0.162396 	 1782 		 985
0.7	0.229305 	 2390 		 1260
0.8	0.112914 	 3756 		 2134
0.9	0.164707 	 4431 		 2340
Sum:	1.784741 	 14376 		 7817



# Comparison of Kornia baseline and fixed version 

* see https://github.com/kornia/kornia/pull/2105/

```
Without fix:                                         With fix:


running experiment: synthetic pi rotation

              error   tentatives        inliers                       error    tentatives       inliers
 90deg     0.516270         6322           4857          90deg     0.000651          6656          6617
180deg     0.674599         5908           3802         180deg     0.001429          4436          4223
270deg     0.454714         6170           3872         270deg     0.001259          6114          5935
   Sum     1.645583        18400          12531            Sum     0.003339         17206         16775


running experiment: bark

              error   tentatives        inliers                       error    tentatives       inliers
img2       1.257790          267             49           img2     1.379990           331            91
img3       2.212071          121             28           img3     1.492706           125            48
img4       1.350093          362            193           img4     1.001385           335           252
img5       0.710791          210             90           img5     0.699119           233           171
img6       1.352123           89             43           img6     0.987482            77            62
 Sum       6.882868         1049            403            Sum     5.560682          1101           624


running experiment: boat

              error   tentatives        inliers                       error    tentatives       inliers
img2       0.625754         1640            425           img2     0.257128          1806           628
img3       0.313360         1155            345           img3     0.387596          1130           417
img4       1.635418          374             90           img4     0.834250           373           106
img5       0.868519          193             42           img5     1.171891           227            59
img6       6.004359           63             16           img6     4.849470            55            16
 Sum       9.447409         3425            918            Sum     7.500336          3591          1226


running experiment: synthetic rescaling lanczos

              error   tentatives        inliers                       error    tentatives       inliers
 0.2       0.337641           46             28            0.2     0.063100            82            69
 0.3       0.270277          219            132            0.3     0.105695           285           192
 0.4       0.316275          597            316            0.4     0.050815           697           565
 0.5       0.191226         1155            622            0.5     0.017677          1311          1160
 0.6       0.162396         1782            985            0.6     0.047982          2065          1616
 0.7       0.229305         2390           1260            0.7     0.050292          2832          2089
 0.8       0.112914         3756           2134            0.8     0.015694          4140          3694
 0.9       0.164707         4431           2340            0.9     0.051795          5233          3939
 Sum       1.784741        14376           7817            Sum     0.403049         16645         13324
```